<a href="https://colab.research.google.com/github/jagvgithub/Classification_Regression_models/blob/main/ModeloPrepagosVehiculo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# paquetes necesarios
!pip install pycaret

# Librerias necearias
from google.colab import files  # carga de datos
import pandas as pd # manipulación de bases de datos
import numpy as np # manipulación de bases de datos
#import gender_guesser.detector as gender
#import gender_guesser.detector as gender # estimación de género a partir del primer y segundo nombre

In [ ]:
from google.colab import files
import pandas as pd

# Cargar el archivo CSV
uploaded = files.upload()

In [ ]:
# Ruta al archivo xlsx en Google Colab
ruta_archivo = '/content/fraude_model.csv'  # Asegúrate de que la ruta sea correcta según la ubicación del archivo cargado

# Leer el archivo xlsx usando pandas
datos = pd.read_csv(ruta_archivo,sep='|', encoding='latin1')
datos=pd.DataFrame(datos)

In [ ]:
columnas = ['UTRNNO', 'TRANS_TYPE','MSGTYPE', 'Cod_Aprovacion','respuesta','POS_ENTRY_MODE','POS_CONDITION_CODE','ECI', 'MCC', 'country_number','Fraude','Testeos']

datos[columnas] = datos[columnas].astype(str)
print(datos.head())

In [ ]:

datos_numericos = datos.select_dtypes(include=['int', 'float'])

# Mostrar los tipos de datos de las columnas seleccionadas
print(datos_numericos.dtypes)

In [ ]:
data=datos.sample(frac=0.75,random_state=786).reset_index(drop=True) # entrenamiento
data_unusen =datos.drop(data.index).reset_index(drop=True) # testeo


# Verificar el tamaño de los conjuntos de entrenamiento y prueba
print("Tamaño del conjunto de entrenamiento:", data.shape)
print("Tamaño del conjunto de prueba:", data_unusen.shape)

In [ ]:
from pycaret.classification import * # la data ya tiene una marcacion de  01
data.reset_index(drop=True, inplace=True)  # Restablecer los índices y eliminar los índices existentes ## debido al concatenado
#exp_clf101 = setup(data=data, target='Fraude', session_id=10001)  # Configurar el experimento de clasificación


exp_clf101 = setup(data=data, target='Fraude', session_id=10001, fold_strategy='stratifiedkfold') # garantizar que todas las clases esten bien representadas


In [ ]:

best_model=compare_models()

In [ ]:

modelo = create_model('lr')

In [ ]:
tuned_modelo = tune_model(modelo)

In [ ]:

print(tuned_modelo)

In [ ]:
help(plot_model) # validacion visual

In [ ]:
plot_model(tuned_modelo,plot='auc')

In [ ]:

# feature importance
plot_model(tuned_modelo,plot='feature')

In [ ]:

## Decision Boundary
plot_model(tuned_modelo, plot = 'boundary')

In [ ]:

# Precision Recall Curve
plot_model(tuned_modelo, plot = 'pr')

In [ ]:
from pycaret.classification import *
predicciones = predict_model(tuned_modelo,data=data_unusen)

predicciones.head()

In [ ]:
plot_model(tuned_modelo,'confusion_matrix')

In [ ]:


# modelo final entrenado con todo el conjunto de datos

final_modelo=finalize_model(tuned_modelo)

In [ ]:
from google.colab import drive
from pycaret.classification import save_model

# Monta Google Drive
drive.mount('/content/drive')

# Guarda el modelo en la carpeta de Google Drive
modelo_a_guardar = final_modelo  # Reemplaza 'final_modelo' con tu modelo
nombre_del_modelo = 'ModeloPrepagoVehiculo'  # Nombre del modelo
ruta_carpeta_drive = '/content/drive/My Drive/Modelos_FA'  # Ruta de la carpeta en Google Drive

# Guarda el modelo
save_model(modelo_a_guardar, f'{ruta_carpeta_drive}/{nombre_del_modelo}')

# Muestra un mensaje de confirmación
print("El modelo se guardó en Google Drive.")
